In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codeT5_embeddings/processed_embeddings/codeT5_small_embed_line_by_line_sum.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511,emb_512,label,sample_id,severity
0,189.393524,-147.649734,-362.955963,139.653641,-85.471199,-66.070679,-474.727875,120.052200,622.299561,116.579880,...,-333.419464,-25.977442,-56.553913,-348.846710,-6.254611,16.088152,160.038864,1,8077,critical
1,3.520596,0.237391,-6.190857,3.774145,1.713395,-4.886091,-11.591867,6.478320,14.700498,2.681563,...,-7.203462,0.416520,-1.725735,-5.898596,-5.929474,2.987580,-6.565398,1,5553,critical
2,3.876191,4.709641,-14.207751,1.015683,-2.791239,-4.798626,-17.614422,5.810671,18.924332,1.923258,...,-14.868880,-5.807928,-1.909955,-13.305037,-2.716194,3.255790,0.641796,1,9341,critical
3,20.053326,-9.636232,-25.695652,0.598159,1.013416,-7.998230,-30.077368,17.036211,28.593739,18.520885,...,-17.841141,1.948726,-4.335785,-17.123833,1.435278,5.188768,3.573848,1,10419,critical
4,188.645142,-144.839706,-355.567810,147.655869,-95.026077,-88.978615,-476.859650,130.700760,622.824158,118.304016,...,-338.199280,-6.570933,-49.001118,-364.383820,-19.460947,19.270901,142.164978,1,12232,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8046
F1_macro: 0.6971
Accuracy: 0.8046
Precision_macro: 0.7823
Recall_macro: 0.6738

Bagging Metrics:
F1_micro: 0.8062
F1_macro: 0.6905
Accuracy: 0.8062
Precision_macro: 0.7985
Recall_macro: 0.6665

XGBoost Metrics:
F1_micro: 0.8132
F1_macro: 0.7380
Accuracy: 0.8132
Precision_macro: 0.7707
Recall_macro: 0.7211


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")


Final Test Results:

Random Forest Metrics:
F1_micro: 0.8133
F1_macro: 0.7219
Precision_macro: 0.7866
Recall_macro: 0.6978
Accuracy: 0.8133
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.94      0.88       814
           1       0.75      0.45      0.56       295

    accuracy                           0.81      1109
   macro avg       0.79      0.70      0.72      1109
weighted avg       0.81      0.81      0.80      1109


Bagging Metrics:
F1_micro: 0.8025
F1_macro: 0.6911
Precision_macro: 0.7832
Recall_macro: 0.6677
Accuracy: 0.8025
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.96      0.88       814
           1       0.76      0.38      0.51       295

    accuracy                           0.80      1109
   macro avg       0.78      0.67      0.69      1109
weighted avg       0.80      0.80      0.78      1109


XGBoost Metrics:
F1_micro: 0.8106
F1_macro: 

In [8]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/data_class/results/codeT5/small/result_codeT5_small_sum.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/data_class/results/codeT5/small/result_codeT5_small_sum.pkl
